In [5]:
! pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 41.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.6 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1.2-cp310-cp310-linux_x86_64.whl size=660545 sha256=ebfc4a701026d2f649e6e6d9320b332bd364283110daff56f976d4667ee57a8e
  Stored in directory: /root/.cache/pip/wheels/70/63/40/4bf006182f942d3516b71bb2ff3b57ccbdb8b2c0ee81882b6e
Successfully built pycuda


In [52]:
import pycuda.autoinit
import pycuda.driver as cuda
import numpy as np
from pycuda.compiler import SourceModule
import time
from prettytable import PrettyTable
import pandas as pd

In [44]:
# Создание рандомных векторов
sizes = [1000, 10000, 100000, 1000000]
n = sizes[0]
a = np.random.randint(low=0, high=500, size=(n)).astype(np.float32)
b = np.random.randint(low=0, high=500, size=(n)).astype(np.float32)
c_gpu_after = np.empty_like(a)

In [45]:
# Сложение векторов на GPU с помощью CUDA

# CUDA ядро
kernel = """
__global__ void vectors_sum(float *a, float *b, float *c, int n) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx < n) {
        c[idx] = a[idx] + b[idx];
    }
}
"""

def gpu_sum_vectors():
  mod = SourceModule(kernel)
  vector_add = mod.get_function("vectors_sum")

  # Выделение памяти на GPU
  a_gpu = cuda.mem_alloc(a.nbytes)
  b_gpu = cuda.mem_alloc(b.nbytes)
  c_gpu = cuda.mem_alloc(c_gpu_after.nbytes)

  # Копирование на GPU векторов, созданных на CPU
  cuda.memcpy_htod(a_gpu, a)
  cuda.memcpy_htod(b_gpu, b)

  # Запуск функции из ядра CUDA
  block_size = 256  # Размер блока
  grid_size = (n + block_size - 1) // block_size  # Количество блоков

  # засекание времени
  start_gpu = time.time()
  vector_add(a_gpu, b_gpu, c_gpu, np.int32(n), block=(block_size, 1, 1), grid=(grid_size, 1))
  cuda.Context.synchronize()
  time_gpu = time.time() - start_gpu

  # Копирование результата с GPU на CPU
  cuda.memcpy_dtoh(c_gpu_after, c_gpu)
  return c_gpu_after, time_gpu


In [46]:
# CPU функция сложения элементов двух векторов через циклы
def cpu_sum_vectors(a, b, n):
  c_cpu = np.zeros(n)
  start_cpu = time.time()
  for i in range(0, n):
    c_cpu[i] = a[i]+b[i]
  time_cpu = time.time() - start_cpu
  return c_cpu, time_cpu

In [50]:
# Результаты

cpu_times = []
gpu_times = []
faster = []
correct = ''
result = []

for i in sizes:
  time_cpu = cpu_sum_vectors(a, b, n)[1]
  time_gpu = gpu_sum_vectors()[1]
  cpu_times.append(time_cpu)
  gpu_times.append(time_gpu)
  faster.append(time_cpu/time_gpu)


  # Матрица С_np посчитана с помощью numpy для проверки корректности результатов
  c_np = a+b
  if np.allclose(cpu_sum_vectors(a, b, n)[0], c_np) and np.allclose(gpu_sum_vectors()[0], c_np):
    correct = 'Посчитано корректно'
  else:
    correct = 'Посчитано неправильно'
  result.append({
      'Размер векторов': i,
      'Время CPU': time_cpu,
      'Время GPU': time_gpu,
      'Ускорение': time_cpu/time_gpu,
      'Правильность подсчета': correct
      })


In [53]:
pd.DataFrame.from_dict(result)

,Размер векторов,Время CPU,Время GPU,Ускорение,Правильность подсчета
0,1000,0.000427,0.000121,3.521654,Посчитано корректно
1,10000,0.000772,0.000045,17.036842,Посчитано корректно
2,100000,0.000465,0.000043,10.893855,Посчитано корректно
3,1000000,0.000473,0.000050,9.447619,Посчитано корректно
